# GSE104284 - Replicated K27ac K4me1 K4me3 mice injury

Logbook: https://docs.google.com/document/d/1vxU3bRWPjKhz4yto3eOQxKDNQQu0xsyi9IUCSsWBS9k/edit#heading=h.44kd47qfiiva


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from IPython.display import display
import pandas as pd

import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy as sp
import os, re
import glob
from tqdm.auto import tqdm
import subprocess, tempfile

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Here and there < 5mln reads
UNDER_5MLN_TRACKS = [
    'GSM2794025_3h_03_Right_B_K27ac_C36PAACXX.8',  
                   'GSM2794037_3h_11_Right_K27ac_C36PAACXX.8',
                  'GSM2794038_3h_11_Right_K4me1_C36PAACXX.8',
                  'GSM2794039_3h_11_Right_K4me3_C36PAACXX.8',
                  'GSM2794040_3h_11_Right_Input_C36PAACXX.8',
                  'GSM2794042_3h_11_Left_K4me1_C36PAACXX.8',
                  'GSM2794044_3h_12_Right_K27ac_C36PAACXX.7',
                  'GSM2794045_3h_12_Right_K4me1_C36PAACXX.7',
                  'GSM2794046_3h_12_Right_K4me3_C36PAACXX.7',
                  'GSM2794047_3h_12_Left_K27ac_C36PAACXX.7',
                  'GSM2794048_3h_12_Left_K4me1_C36PAACXX.7',
                  'GSM2794049_3h_12_Left_K4me3_C36PAACXX.7',
                  'GSM2794072_C4KGFACXX.1.10hr_06_Right_K4me3',
                  'GSM2794086_C36UVACXX.3.24hr_02_Left_K27ac',
                  'GSM2794205_C3A8DACXX140504.1.7d_02_Right_K27ac',
                  'GSM2794214_C3A8DACXX140504.5.7d_03_Right_K4me1',
                   'GSM2794222_C3A8DACXX140504.7.7d_04_Right_K4me3',
                   'GSM2794225_C3A8DACXX140504.8.7d_04_Right_K4me3',
                   'GSM2794235_C3A8DACXX140504.7.7d_04B_Right_K27ac',
                   'GSM2794239_C3A8DACXX140504.8.7d_04B_Right_K27ac',
                   'GSM2794244_C3A8DACXX140504.7.7d_04B_Left_K4me1',
                   'GSM2794245_C3A8DACXX140504.7.7d_04B_Left_K4me3',
                   'GSM2794248_C3A8DACXX140504.8.7d_04B_Left_K4me1',
                   'GSM2794249_C3A8DACXX140504.8.7d_04B_Left_K4me3',
                   'GSM2794280_C4KGFACXX.8.14d_04_Right_K4me1',
                   'GSM2794288_C4KGFACXX.8.14d_05_Right_K4me3',
                  ]

def ok(f):
    for bt in UNDER_5MLN_TRACKS:
        if bt in f:
            return False
    return True

# Downloaded GEO peaks

In [ ]:
GEO_FOLDER='/mnt/stripe/bio/raw-data/geo-samples/GSE104284/geo'

dfg = pd.DataFrame(columns=['gsm', 'time', 'direction', 'modification', 'replicate', 
                            'level', 'file', 'peaks', 'length'])
for file in tqdm(glob.glob(GEO_FOLDER + '/*.*')):
    if 'Input' in file or not ok(file):
        continue
    gsm = re.sub('_.*', '', os.path.basename(file))
    time = re.sub(f'(.*Aguilar_)|(_[0-9a-zA-Z]+_.*)', '', os.path.basename(file))
    direction = next((d for d in ['Left', 'Right'] if d in os.path.basename(file)), None)
    modification = next((m for m in ['K4me1', 'K4me3', 'K27ac'] if m in os.path.basename(file)), None)        
    replicate = re.sub(f'(.*{time}_)|(_{direction}.*)', '', os.path.basename(file))
    out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
    if out[0].strip() != '':
        peaks, length = out[0].split(' ') 
    else:
        peaks, length = 0, 0
    dfg.loc[len(dfg)] = (gsm, time, direction, modification, replicate, 'geo', file, peaks, length)

# display(dfg.head())    
    
dfg['timesrt'] = ["{:0>3d}".format(int(re.match('[0-9]+', t)[0]) * 24) if 
                   t.endswith('d') else "{:0>3d}".format(int(re.match('[0-9]+', t)[0])) 
                  for t in dfg['time'] ]
        
# Fix types
dfg['peaks'] = dfg['peaks'].astype(int)
dfg['length'] = dfg['length'].astype(int)

geo_gsms = set(dfg['gsm'])

In [ ]:
dfg['mdt'] = dfg['modification'] + '_' + dfg['direction'] + '_' + dfg['timesrt']
dfg_mean = dfg.groupby(['mdt', 'level'])['peaks'].mean().reset_index().sort_values(by=['mdt', 'level'])
dfg_std = dfg.groupby(['mdt', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['mdt', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfg_mean['level'])):
    fig.add_trace(go.Scatter(x=dfg_mean.loc[dfg_mean['level']==l]['mdt'], 
                             y=dfg_mean.loc[dfg_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfg_std.loc[dfg_std['level']==l]['mdt'], 
                             y=dfg_std.loc[dfg_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

# Overlapping

In [ ]:
from pathlib import Path
import downstream.bed_metrics as bm

def overlap_with_name_level(overlaps, m, l):
    dfo = overlaps[(m, l)].melt(value_name='overlap')
    dfo['modification'] = m
    dfo['level'] = l
    return dfo

def show_overlap(df):
    levels = sorted(set(df['level']))
    overlaps = {}
    for m in set(df['modification']):
        for l in levels:
            print('Processing', m, l)
            files = df.loc[np.logical_and(df['modification'] == m, df['level'] == l)]['file']
            paths = [Path(f) for f in files]
            df_path = f'/tmp/overlap_{m}_{l}.tsv'
            overlaps[(m, l)] = bm.load_or_build_metrics_table(paths, paths, Path(df_path), jaccard=False)

    dfo = pd.concat([overlap_with_name_level(overlaps, m, l) for (m, l) in overlaps])        
    dfo_mean = dfo.groupby(['modification', 'level'])['overlap'].mean().reset_index().sort_values(
        by=['modification'])
    dfo_std = dfo.groupby(['modification', 'level'])['overlap'].std().reset_index().fillna(0).sort_values(
        by=['modification']) 
    fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Overlap")))
    for l in levels:
        fig.add_trace(go.Scatter(x=dfo_mean.loc[dfo_mean['level']==l]['modification'], 
                                 y=dfo_mean.loc[dfo_mean['level']==l]['overlap'], 
                                 name=f"{l} mean", line_shape='linear'))
        fig.add_trace(go.Scatter(x=dfo_std.loc[dfo_std['level']==l]['modification'], 
                                 y=dfo_std.loc[dfo_std['level']==l]['overlap'], 
                                 name=f"{l} std", line_shape='linear', 
                                 line=dict(dash='dot')))
    fig.show()

In [ ]:
# Show me overlaps   
geolevels2process = set(['geo'])
# show_overlap(dfg.loc[[l in geolevels2process for l in dfg['level']]])

# MACS2

In [ ]:
MACS2_FOLDER='/mnt/stripe/bio/raw-data/geo-samples/GSE104284/macs2'
MACS2_LEVELS = ['broad_0.05']

dfm = pd.DataFrame(columns=['gsm', 'time', 'direction', 'modification', 'replicate', 
                            'level', 'file', 'peaks', 'length'])
for file in tqdm(glob.glob(MACS2_FOLDER + '/*.*Peak')):
    if 'gapped' in file or 'Input' in file or not ok(file):
        continue
    level = next((l for l in MACS2_LEVELS if f'_{l}' in file), None) # 
    if level:
        gsm = re.sub('_.*', '', os.path.basename(file))
        if gsm not in geo_gsms:
            continue  # Ignore gsms not processed by authors
        time = re.sub(f'({gsm}_([^_]*.\.)?)|(_[0-9a-zA-Z]+_.*)', '', os.path.basename(file))
        direction = next((d for d in ['Left', 'Right'] if d in os.path.basename(file)), None)
        modification = next((m for m in ['K4me1', 'K4me3', 'K27ac'] if m in os.path.basename(file)), None)        
        replicate = re.sub(f'(.*{time}_)|(_{direction}.*)', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfm.loc[len(dfm)] = (gsm, time, direction, modification, replicate, f'macs2 {level}', file, peaks, length)

dfm['timesrt'] = ["{:0>3d}".format(int(re.match('[0-9]+', t)[0]) * 24) if 
                   t.endswith('d') else "{:0>3d}".format(int(re.match('[0-9]+', t)[0])) 
                  for t in dfm['time'] ]
        
# Fix types
dfm['peaks'] = dfm['peaks'].astype(int)
dfm['length'] = dfm['length'].astype(int)

In [ ]:
dfm['mdt'] = dfm['modification'] + '_' + dfm['direction'] + '_' + dfm['timesrt']
dfm_mean = dfm.groupby(['mdt', 'level'])['peaks'].mean().reset_index().sort_values(by=['mdt', 'level'])
dfm_std = dfm.groupby(['mdt', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['mdt', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfm_mean['level'])):
    fig.add_trace(go.Scatter(x=dfm_mean.loc[dfm_mean['level']==l]['mdt'], 
                             y=dfm_mean.loc[dfm_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfm_std.loc[dfm_std['level']==l]['mdt'], 
                             y=dfm_std.loc[dfm_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
# Show me overlaps   
macs2levels2process = set(['macs2 broad_0.05'])
# show_overlap(dfm.loc[[l in macs2levels2process for l in dfm['level']]])

# SICER

In [ ]:
SICER_FOLDER='/mnt/stripe/bio/raw-data/geo-samples/GSE104284/sicer'
SICER_LEVELS = ['FDR0.01']

dfsc = pd.DataFrame(columns=['gsm', 'time', 'direction', 'modification', 'replicate', 
                            'level', 'file', 'peaks', 'length'])
for file in tqdm(glob.glob(SICER_FOLDER + '/*islands-summary*')):
    if 'Input' in file or not ok(file):
        continue
    level = next((l for l in SICER_LEVELS if l in file), None) # 
    if level:
        gsm = re.sub('_.*', '', os.path.basename(file))
        if gsm not in geo_gsms:
            continue  # Ignore gsms not processed by authors
        time = re.sub(f'({gsm}_([^_]*.\.)?)|(_[0-9a-zA-Z]+_.*)', '', os.path.basename(file))
        direction = next((d for d in ['Left', 'Right'] if d in os.path.basename(file)), None)
        modification = next((m for m in ['K4me1', 'K4me3', 'K27ac'] if m in os.path.basename(file)), None)        
        replicate = re.sub(f'(.*{time}_)|(_{direction}.*)', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfsc.loc[len(dfsc)] = (gsm, time, direction, modification, replicate, f'sicer {level}', file, peaks, length)

dfsc['timesrt'] = ["{:0>3d}".format(int(re.match('[0-9]+', t)[0]) * 24) if 
                   t.endswith('d') else "{:0>3d}".format(int(re.match('[0-9]+', t)[0])) 
                  for t in dfsc['time'] ]
        
# Fix types
dfsc['peaks'] = dfsc['peaks'].astype(int)
dfsc['length'] = dfsc['length'].astype(int)

In [ ]:
dfsc['mdt'] = dfsc['modification'] + '_' + dfsc['direction'] + '_' + dfsc['timesrt']
dfsc_mean = dfsc.groupby(['mdt', 'level'])['peaks'].mean().reset_index().sort_values(by=['mdt', 'level'])
dfsc_std = dfsc.groupby(['mdt', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['mdt', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfsc_mean['level'])):
    fig.add_trace(go.Scatter(x=dfsc_mean.loc[dfsc_mean['level']==l]['mdt'], 
                             y=dfsc_mean.loc[dfsc_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfsc_std.loc[dfsc_std['level']==l]['mdt'], 
                             y=dfsc_std.loc[dfsc_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
# Show me overlaps   
sicerlevels2process = set(['sicer FDR0.01'])
# show_overlap(dfsc.loc[[l in sicerlevels2process for l in dfsc['level']]])

# SPAN automated markup

In [ ]:
# %%bash
# # Bash commands to create markup based on consensus
# DIR=/mnt/stripe/bio/raw-data/geo-samples/GSE104284
# T=$'\t'; 

# for M in K4me1 K4me3 K27ac; do
#     echo "Processing $M"
#     OUT=${DIR}/intersect_${M}.tsv;

#     printf %s "chr${T}start${T}end" > ${OUT}; 
#     FILES=(); 
#     for F in $(find ${DIR}/macs2/ -name "*${M}*.broadPeak"); do 
#         FILES+=("$F"); 
#         printf %s "${T}${F}" >> ${OUT}; 
#     done; 
#     echo >> ${OUT};
#     bedtools multiinter -i "${FILES[@]}" |\
#         bedtools merge -c $(seq -s, 6 1 $((${#FILES[@]} + 5))) -o max |\
#         awk '{if (NR > 1) printf("\n"); printf("%s\t%s\t%s", $1, $2, $3); for (i=4; i<=NF; i++) printf("\t%d", int($i)); }' >> ${OUT};

#     # Find out regions confirmed by at least 50% of files
#     CONS=$((${#FILES[@]}/2))
#     cat ${OUT} | awk -v OFS='\t' CONS=${CONS} 'sum=0; for (i=4; i<=NF; i++){sum+= $i}; if (sum>=CONS){print($1,$2,$3)}' > ${DIR}/intersect_all_${M}.bed

#     # Find any of the peaks to get scores
#     F=$(find ${DIR}/macs2/ -name "*${M}*.broadPeak" | head -n 1);
#     bedtools intersect -a ${F} -b ${DIR}/intersect_all_${M}.bed -wa > ${DIR}/intersect_all_${M}.broadPeak
# done

In [ ]:
# # Sample number of consensus peak with p-value stratification
# from sklearn.utils import shuffle

# wdir = '/mnt/stripe/bio/raw-data/geo-samples/GSE104284'
# for m in ['K4me1', 'K4me3', 'K27ac']:
#     print('Processing', m)
#     idf = pd.read_csv(f'{wdir}/intersect_all_{m}.broadPeak',
#                       names=['chr', 'start', 'end', 'name', 'score', 'strand', 'summitfc', 'mlogp', 'mlogq', 'summit'], 
#                       sep='\t')
#     idf.sort_values(by=['mlogq'], ascending=False, inplace=True)

#     markup_size = 2000
#     peaks_file = f'{wdir}/peaks_{m}_{markup_size}.bed'
#     step = int(len(idf) / markup_size)
#     markup_df = idf.loc[[i % step == 0 for i in range(len(idf))]]
#     shuffle(markup_df[['chr', 'start', 'end']]).to_csv(peaks_file, sep='\t', header=None, index=False)
#     print(f'Saved {markup_size} peaks stratified by p-value to {peaks_file}')

# print('Done')

In [ ]:
# %%bash
# DIR=/mnt/stripe/bio/raw-data/geo-samples/GSE104284

# for M in K4me1 K4me3 K27ac; do
#     echo "Processing $M"

#     # Total 2000 peaks 1000,500,500
#     # peaks
#     head -n 1000 ${DIR}/peaks_${M}_2000.bed | while read -r LINE; do \
#         echo "$LINE" | awk -v OFS='\t' '{print $1,$2,$3,"peaks"}'; \
#     done > ${DIR}/${M}_labels.bed;

#     # peakStart
#     head -n 1500 ${DIR}/peaks_${M}_2000.bed | tail -n 500 | while read -r LINE; do \
#         echo "$LINE" | awk '{ printf("%s\t%d\t%d\t%s\n", $1,$2-1000,($2+$3)/2 - 1,"peakStart")}'; \
#     done >> ${DIR}/${M}_labels.bed;

#     # peakEnd
#     head -n 2000 ${DIR}/peaks_${M}_2000.bed | tail -n 500 | while read -r LINE; do \
#         echo "$LINE" | awk '{printf("%s\t%d\t%d\t%s\n", $1,($2+$3)/2 + 1,$3+1000,"peakEnd")}'; \
#     done >> ${DIR}/${M}_labels.bed;

#     # extended markup
#     cat ${DIR}/${M}_labels.bed | while read -r LINE; do \
#         echo "$LINE" | awk '{print($1,$2-2000,$3+2000)}'; \
#     done > ${DIR}/${M}_labels_ext.bed;
# done

# SPAN

In [ ]:
SPAN_FOLDER='/mnt/stripe/bio/raw-data/geo-samples/GSE104284/span'
# SPAN_LEVELS = ['200_0.01_5', '200_1E-6_5', 'tuned']
SPAN_LEVELS = ['tuned']

dfs = pd.DataFrame(columns=['gsm', 'time', 'direction', 'modification', 'replicate', 
                            'level', 'file', 'peaks', 'length'])
for file in tqdm(glob.glob(SPAN_FOLDER + '/*.peak')):
    if 'gapped' in file or 'Input' in file or not ok(file):
        continue
    level = next((l for l in SPAN_LEVELS if f'_{l}' in file), None) # 
    if level:
        gsm = re.sub('_.*', '', os.path.basename(file))
        if gsm not in geo_gsms:
            continue  # Ignore gsms not processed by authors
        time = re.sub(f'({gsm}_([^_]*.\.)?)|(_[0-9a-zA-Z]+_.*)', '', os.path.basename(file))
        direction = next((d for d in ['Left', 'Right'] if d in os.path.basename(file)), None)
        modification = next((m for m in ['K4me1', 'K4me3', 'K27ac'] if m in os.path.basename(file)), None)        
        replicate = re.sub(f'(.*{time}_)|(_{direction}.*)', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfs.loc[len(dfs)] = (gsm, time, direction, modification, replicate, f'span {level}', file, peaks, length)

dfs['timesrt'] = ["{:0>3d}".format(int(re.match('[0-9]+', t)[0]) * 24) if 
                   t.endswith('d') else "{:0>3d}".format(int(re.match('[0-9]+', t)[0])) 
                  for t in dfs['time'] ]
        
# Fix types
dfs['peaks'] = dfs['peaks'].astype(int)
dfs['length'] = dfs['length'].astype(int)

In [ ]:
dfs['mdt'] = dfs['modification'] + '_' + dfs['direction'] + '_' + dfs['timesrt']
dfs_mean = dfs.groupby(['mdt', 'level'])['peaks'].mean().reset_index().sort_values(by=['mdt', 'level'])
dfs_std = dfs.groupby(['mdt', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['mdt', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfs_mean['level'])):
    fig.add_trace(go.Scatter(x=dfs_mean.loc[dfs_mean['level']==l]['mdt'], 
                             y=dfs_mean.loc[dfs_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfs_std.loc[dfs_std['level']==l]['mdt'], 
                             y=dfs_std.loc[dfs_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
# Show me overlaps   
spanlevels2process = set(['span tuned'])
# show_overlap(dfs.loc[[l in spanlevels2process for l in dfs['level']]])

# Summary

In [ ]:
dfa = pd.concat([dfg.loc[[l in geolevels2process for l in dfg['level']]],
                 dfm.loc[[l in macs2levels2process for l in dfm['level']]],
                 dfsc.loc[[l in sicerlevels2process for l in dfsc['level']]],
                 dfs.loc[[l in spanlevels2process for l in dfs['level']]]])

In [ ]:
dfa['mdt'] = dfa['modification'] + '_' + dfa['direction'] + '_' + dfa['timesrt']
dfa_mean = dfa.groupby(['mdt', 'level'])['peaks'].mean().reset_index().sort_values(by=['mdt', 'level'])
dfa_std = dfa.groupby(['mdt', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['mdt', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfa_mean['level'])):
    fig.add_trace(go.Scatter(x=dfa_mean.loc[dfa_mean['level']==l]['mdt'], 
                             y=dfa_mean.loc[dfa_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfa_std.loc[dfa_std['level']==l]['mdt'], 
                             y=dfa_std.loc[dfa_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
dfa_mean = dfa.groupby(['modification', 'level'])['peaks'].mean().reset_index().sort_values(
    by=['modification', 'level'])
dfa_std = dfa.groupby(['modification', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(
    by=['modification', 'level'])

In [ ]:
fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfa_mean['level'])):
    fig.add_trace(go.Scatter(x=dfa_mean.loc[dfa_mean['level']==l]['modification'], 
                             y=dfa_mean.loc[dfa_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfa_std.loc[dfa_std['level']==l]['modification'], 
                             y=dfa_std.loc[dfa_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
dfa['avg_length'] = dfa['length'] / dfa['peaks']
dfa.loc[~np.isfinite(dfa["avg_length"]), "avg_length"] = 0.0

# Groups analysis

In [ ]:
def plot_data_cells(df, cid, value, description):
    cids = sorted(set(df[cid]))
    axs = {}
    total = len(cids) * 3
    fig = plt.figure(figsize=(int(total * .75), 4))
    offset = 0
    for m in ['K27ac', 'K4me1', 'K4me3']:
        data = df.loc[df['modification'] == m].sort_values(by=[cid])
        xlabels = []
        for c in data[cid]:
            if c not in xlabels:
                xlabels.append(c)
        w = len(cids)
        ax = plt.subplot2grid((1, total), (0, offset), colspan=w)

        sns.barplot(data=data, 
                     x=cid, y=value,
                     ci="sd", capsize=.2, errwidth=2,
                     edgecolor="black",
                     ax = ax)

        sns.swarmplot(data=data,
                      x=cid, y=value,
                      size=1,
                      color="black",
                      alpha=0.5,
                      ax = ax)
        ax.legend().set_visible(False)
        axs[ax] = plt.ylim()
        if offset > 0:
            ax.get_yaxis().set_ticklabels([])
            ax.set_ylabel('')
        else:
            ax.set_ylabel(description)
        
        offset += w
        ax.set_xlabel('')
        ax.set_title(m)
        plt.xticks(range(0, len(xlabels)), xlabels, rotation=45)
            
    ymin = np.min([v[0] for v in axs.values()])
    ymax = np.max([v[1] for v in axs.values()])
 
    for ax in axs.keys():
        ax.set_ylim(bottom = ymin, top = ymax)
    plt.tight_layout()

In [ ]:
plot_data_cells(dfa, 'level', 'peaks', 'Peaks')
plt.show()
plot_data_cells(dfa, 'level', 'avg_length', 'Average peak length')
plt.show()

In [ ]:
dfa['ld'] = dfa['level'] + '_' + dfa['direction']
plot_data_cells(dfa, 'ld', 'peaks', 'Peaks')
plt.show()
plot_data_cells(dfa, 'ld', 'avg_length', 'Average peak length')
plt.show()

# Overlap

In [ ]:
show_overlap(dfa)

In [ ]:
def compute_overlap(df):
    # Compute overlaps
    dft = pd.DataFrame(columns=['id', 'modification', 'level', 'direction', 'overlap'])
    levels = sorted(set(df['level']))
    for m in ['K27ac', 'K4me1', 'K4me3']:
        for l in levels:
            for d in ['Left', 'Right']:
                paths = [Path(f) for f in df.loc[np.logical_and(df['modification']==m, 
                    np.logical_and(df['level']==l, df['direction']==d))]['file']]
                df_path = f'/tmp/overlap_{m}_{l}_{d}.tsv'
                mt = bm.load_or_build_metrics_table(paths, paths, Path(df_path),
                                                    jaccard=False,
                                                    threads=30)
                for row in mt.index:
                    for col in mt.columns:
                        overlap = mt.loc[row][col]
                        dft.loc[len(dft)] = (f'{row}@{col}', m, l, d, overlap)
    return dft

In [ ]:
df_overlap = compute_overlap(dfa)
df_overlap['ld'] = df_overlap['level'] + '_' + df_overlap['direction']
plot_data_cells(df_overlap, 'ld', 'overlap', 'Overlaps')